In [1]:
# Imports
import ffmpeg
import pandas as pd
import geopandas
import datetime
import shapely
import warnings
import numpy as np

import video_transect, misc

In [2]:
# Filter future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Paths
csv_file = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect1/2022-03-29_125003_MassimalTrack.csv'
video_dir = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect1'
#image_dir = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/ExtractedImages'
#gpk_file = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/test_output.gpkg'

In [4]:
# Read file, get geodataframe
gdf = video_transect.track_csv_to_geodataframe(csv_file)

In [5]:
gdf

,Lat,Lng,Time,TimeDiffSec,geometry
0,67.227312,14.625156,2022-03-29 10:50:04.089000+00:00,0.000,POINT (14.62516 67.22731)
1,67.227312,14.625156,2022-03-29 10:50:04.089000+00:00,0.000,POINT (14.62516 67.22731)
2,67.227312,14.625156,2022-03-29 10:50:05.098000+00:00,1.009,POINT (14.62516 67.22731)
3,67.227312,14.625156,2022-03-29 10:50:06.094000+00:00,2.005,POINT (14.62516 67.22731)
4,67.227312,14.625156,2022-03-29 10:50:07.098000+00:00,3.009,POINT (14.62516 67.22731)
...,...,...,...,...,...
3719,67.227312,14.625167,2022-03-29 11:52:02.075000+00:00,3717.986,POINT (14.62517 67.22731)
3720,67.227327,14.625210,2022-03-29 11:52:39.451000+00:00,3755.362,POINT (14.62521 67.22733)
3721,67.227328,14.625213,2022-03-29 11:52:40.338000+00:00,3756.249,POINT (14.62521 67.22733)
3722,67.227320,14.625202,2022-03-29 11:52:41.338000+00:00,3757.249,POINT (14.62520 67.22732)


In [7]:
# Get list of video files, insert into dataframe
video_files = misc.file_pattern_search(video_dir, '*.[Mm][Pp]4')    # Use brackets to find both .mp4 and .MP4 files

In [21]:
video_duration = np.zeros(len(video_files))
video_frame_rate = np.zeros(len(video_files))

for ii,file in enumerate(video_files):
    probe_data = ffmpeg.probe(file)
    # Note that there are multiple streams. Stream 0 is the video.
#     video_data['StartTime'][ii] = pd.to_datetime(probe_data['streams'][0]['tags']['creation_time'])
    video_duration[ii] = np.float64(probe_data['streams'][0]['duration'])
    video_frame_rate[ii] = eval(probe_data['streams'][0]['avg_frame_rate'])
    
#     print('********')
#     print(file)
#     for key, value in probe_data['streams'][0].items():
#        print(key + ': ' + str(value))

In [29]:
# Calculate start and end times
video_start_time = np.zeros(len(video_duration))
video_start_time[1:] = np.cumsum(video_duration[0:-1])
video_end_time = video_start_time + video_duration


In [27]:
video_start_time

array([   0.   ,  532.532, 1065.064, 1597.596, 2130.128, 2662.66 ])

In [28]:
video_end_time

array([ 532.532   , 1065.064   , 1597.596   , 2130.128   , 2662.66    ,
       2911.441867])

In [34]:
# Boolean indexing for finding right file
for ii, file in enumerate(video_files):
    ind = (gdf['TimeDiffSec'] >= video_start_time[ii]) & (gdf['TimeDiffSec'] < video_end_time[ii])
    #gdf['VideoFile']

In [35]:
ind

0       False
1       False
2       False
3       False
4       False
        ...  
3719    False
3720    False
3721    False
3722    False
3723    False
Name: TimeDiffSec, Length: 3724, dtype: bool

In [ ]:
tmp = gdf['']